# README

### Purpose of this notebook
- Create dataframe for applicants' info.

### Steps
1. Concatenate all raw csv files into one csv file.
2. Extract columns and create dataframe from the merged data sheet.
3. Simple EDA and perform null value check in the dataframe.
    - Handle outliers

# Import Library

In [ ]:
import pandas as pd
import numpy as np

from importlib import reload

# Utility variable
import sys
sys.path.insert(0, '../..')

# var
import var.path as P
import var.var as V

# utils
import utils.data as D

# Applicants after year 110 (inclusive)
- Extract data from data sheet.

## Read raw data and preprocess

In [ ]:
df = pd.read_csv(P.FP_ALL_DATA_SHEET_CSV)
df.shape

In [ ]:
df.columns.to_list()

## Extract desired data

In [ ]:
col_year = "# The content is removed due to confidential concerns."
col_id = "# The content is removed due to confidential concerns."
col_name = "# The content is removed due to confidential concerns."
col_summarized_talent = "# The content is removed due to confidential concerns."
col_main_talent = "# The content is removed due to confidential concerns."
col_secondary_talent = "# The content is removed due to confidential concerns."
col_special_background = "# The content is removed due to confidential concerns."
col_school = "# The content is removed due to confidential concerns."

In [ ]:
df_applicants_data_list = []

"""
First, create a list of dictionary (row data).
Then, create the dataframe with the list of data.
NEVER append row data inside a for loop, which is a common cause of the poor performance.
(At each iteration, a new dataframe would be created. What a waste!)
"""

for idx, row in df.iterrows():
    row_data = {
        'year': row[col_year],
        'id': row[col_id],
        'name': row[col_name],
        'summarized_talent': row[col_summarized_talent],
        'main_talent': row[col_main_talent],
        'secondary_talent': row[col_secondary_talent],
        'special_background': row[col_special_background],
        'school': row[col_school],
    }
        
    df_applicants_data_list.append(row_data)

# Applicants before year 109 (inclusive)
- Extract data from comments.

## Read raw data and preprocess

In [ ]:
df = pd.read_csv(P.FP_ALL_COMMENT_CSV)
df.shape

In [ ]:
df.columns.to_list()

In [ ]:
col_year = "# The content is removed due to confidential concerns."
col_id = "# The content is removed due to confidential concerns."
col_name = "# The content is removed due to confidential concerns."
col_summarized_talent = "# The content is removed due to confidential concerns."
col_main_talent = "# The content is removed due to confidential concerns."
col_special_background = "# The content is removed due to confidential concerns."
col_school = "# The content is removed due to confidential concerns."

In [ ]:
"""
Keep collection row data entry.
"""

for idx, row in df.iterrows():
    row_data = {
        'year': row[col_year],
        'id': row[col_id],
        'name': row[col_name],
        'summarized_talent': row[col_summarized_talent],
        'main_talent': row[col_main_talent],
        'special_background': row[col_special_background],
        'school': row[col_school],
    }
    
    if row_data['year'] <= 109:
        df_applicants_data_list.append(row_data)

In [ ]:
## Create dataframe                
df_applicants = pd.DataFrame(df_applicants_data_list)

In [ ]:
df_applicants.sort_values(by=['year', 'id'], inplace=True)
df_applicants.reset_index(drop=True, inplace=True)

In [ ]:
df_applicants.head()

### Map talent to predefined field
- Only focus on summarized talent label after year 109

In [ ]:
def summarized_talent_matching(s_talent):
    if type(s_talent) == float and np.isnan(s_talent):
        return "無"
    
    field = ""
    for talent in V.TALENT_LIST:
        if talent in s_talent:
            field = V.TALENT_TO_FIELD_MAPPING[talent]
            break
    
    if field == "":
        print(s_talent)
    
    return field

In [ ]:
df_applicants["talent_field"] = df_applicants.summarized_talent.apply(summarized_talent_matching)

In [ ]:
df_applicants["talent_field"].value_counts()

In [ ]:
for col in df_applicants:
    print(df_applicants[col].isna().value_counts())
    print('-'*50)

## Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_applicants_train, df_applicants_test = train_test_split(
    df_applicants, test_size=0.1, random_state=42
)

In [ ]:
df_applicants_train["train_or_test"] = "train"
df_applicants_test["train_or_test"] = "test" ## Actually is validation set

In [ ]:
df_applicants = pd.concat([df_applicants_train, df_applicants_test])

In [ ]:
test_tuple = [
    "# The content is removed due to confidential concerns."
]

In [ ]:
def force_test_data(row):
    _year = row['year']
    _id = row['id']
    
    if (_year, _id) in test_tuple:
        return "test"
    
    return row['train_or_test']

In [ ]:
df_applicants['train_or_test'] = df_applicants.apply(force_test_data, axis=1)

In [ ]:
df_applicants['train_or_test'].value_counts()

## Sort the dataframe

In [ ]:
df_applicants.sort_values(by=['year', 'id'], inplace=True)
df_applicants.reset_index(drop=True, inplace=True)

In [ ]:
df_applicants.head()

In [ ]:
df_applicants.tail()

## Save to dataframe

In [ ]:
D.write_df_applicants(df_applicants, file='csv')
D.write_df_applicants(df_applicants, file='pkl')